In [1]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.in_out.basics import read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, \
average_per_class, pclouds_with_zero_mean_in_unit_sphere

from sklearn.svm import LinearSVC
from general_tools.simpletons import sort_dict_by_key, sort_dict_by_val

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '40'

In [5]:
# NIPS - CONV
# experiment_name = 'all_snc_rotated_conv_arch_2048pts_chamfer'
# top_data_dir = '/scr/optas/DATA/nips/nn_models/vanilla_ae/'


# no_bnorm - deep- not wide (new architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc'
experiment_name ='all_snc_rotated_deeper_mlp_arch_1_pc2048pts_chamfer'
# - it requires c approx 100

# bnorm on encoder - deep- not wide (old architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name ='all_snc_rotated_deeper_mlp_arch_1_pc_bnorm_on_encoder_old_code2048pts_chamfer'
# - >  seems to work with both big and small c

# no bnorm - deep- and wide (new architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'all_snc_rotated_deeper_wider_mlp_arch_1_pc2048pts_chamfer'
# 2K epochs (200, 0.2, True, 'squared_hinge'), 0.7947793757649938),
# 1Kepochs [((80, 2, True, 'squared_hinge'), 0.79456640146878821),

 
# no bnorm  - deep- not wide (old architecture)
# top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
# experiment_name = 'all_snc_rotated_deeper_mlp_arch_1_pc_no_bnorm_old_code2048pts_chamfer'
# > epoch 600smething was around 81.
# > epoch 850 (40, 2, True, 'squared_hinge'), 0.80518268053855557)

# top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
# experiment_name = 'no_conv_deeper_snc_rotated2048pts_chamfer'

experiment_name = '1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer'
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/'

In [6]:
train_dir = osp.join(top_data_dir, experiment_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))

if conf.train_dir != train_dir: # added to address moving training-folder to other location
    conf.train_dir = train_dir
    conf.save(osp.join(conf.train_dir, 'configuration'))

print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = True

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
               experiment_name: 1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None


In [12]:
# tf.logging.set_verbosity(tf.logging.WARN)

In [16]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
# last_epoch = saved_epochs[-1]
last_epoch = 2000
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

INFO:tensorflow:Restoring parameters from /orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer/models.ckpt-2000
Model restored in epoch 2000.


In [11]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [17]:
verbose = True
n_threads = 20
put_in_usphere = True
pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)
if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, labels = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [18]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [22]:
intercept_scaling_space = [0.1, 0.2, 0.4, 0.5, 1, 2, 3, 4]
dual_space = [False, True]
loss_space = ['hinge', 'squared_hinge']
c_space = [0.6, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
c_space = [0.45, 0.50, 0.55]

# c_space = [0.4]
# intercept_scaling_space = [1]
# dual_space = [False]
# loss_space= ['squared_hinge']

all_scores = dict()

for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual)
                    lsvc.fit(train_latent, cids)
                    print c, intercept, dual, loss
                    score = average_per_class(lsvc, test_latent, cids_)                
                    print score
                    all_scores[(c, intercept, dual, loss)] = score
                except:
                    continue

0.45 0.1 False squared_hinge
0.833186352509
0.45 0.1 True hinge
0.816789167687
0.45 0.1 True squared_hinge
0.825267747858
0.45 0.2 False squared_hinge
0.833436352509
0.45 0.2 True hinge
0.818820379437
0.45 0.2 True squared_hinge
0.817672888617
0.45 0.4 False squared_hinge
0.833686352509
0.45 0.4 True hinge
0.819789167687
0.45 0.4 True squared_hinge
0.832575887393
0.45 0.5 False squared_hinge
0.834645654835
0.45 0.5 True hinge
0.816585679315
0.45 0.5 True squared_hinge
0.82892870257
0.45 1 False squared_hinge
0.832145654835
0.45 1 True hinge
0.782756119951
0.45 1 True squared_hinge
0.805804773562
0.45 2 False squared_hinge
0.833645654835
0.45 2 True hinge
0.789539167687
0.45 2 True squared_hinge
0.817058445532
0.45 3 False squared_hinge
0.833145654835
0.45 3 True hinge
0.787143818849
0.45 3 True squared_hinge
0.816110771114
0.45 4 False squared_hinge
0.832895654835
0.45 4 True hinge
0.813988678091
0.45 4 True squared_hinge
0.800860771114
0.5 0.1 False squared_hinge
0.832645654835
0.5 0.

In [21]:
sort_dict_by_val(all_scores, reverse=True)

[((0.35, 0.1, False, 'squared_hinge'), 0.83534914320685449),
 ((0.2, 0.5, False, 'squared_hinge'), 0.8353240514075887),
 ((0.1, 3, False, 'squared_hinge'), 0.83524265605875159),
 ((0.1, 2, False, 'squared_hinge'), 0.83524265605875159),
 ((0.15, 0.1, False, 'squared_hinge'), 0.83457405140758867),
 ((0.15, 0.5, False, 'squared_hinge'), 0.83457405140758867),
 ((0.25, 3, True, 'squared_hinge'), 0.83449265605875156),
 ((0.4, 1, False, 'squared_hinge'), 0.83443635250917991),
 ((0.4, 0.1, False, 'squared_hinge'), 0.83443635250917991),
 ((0.15, 4, False, 'squared_hinge'), 0.8343240514075887),
 ((0.15, 0.4, False, 'squared_hinge'), 0.8343240514075887),
 ((0.15, 1, False, 'squared_hinge'), 0.8342833537331702),
 ((0.15, 0.2, False, 'squared_hinge'), 0.8342833537331702),
 ((0.1, 4, False, 'squared_hinge'), 0.8342833537331702),
 ((0.15, 2, False, 'squared_hinge'), 0.8342833537331702),
 ((0.15, 3, False, 'squared_hinge'), 0.83403335373317022),
 ((0.1, 1, False, 'squared_hinge'), 0.83399265605875139)

In [ ]:
# New stuff for ICLR
# 1.  Chamfer
# encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
# decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}

# -> @ 980 epochs of training: best was on model-net40:
# 0.6 1 True hinge
# 0.840083537332


# -> @ 1620 epochs of training: best was on model-net40:
# 0.4 1 False squared_hinge
# 0.843783353733
# at @ 2000 epochs of training: best was on model-net40:
# (0.3, 0.2, False, 'squared_hinge'), 0.84087423500612002)


In [ ]:
# FROM NIPS
# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674


# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83